In [43]:
# %% Import libraries
import pandas as pd
import numpy as np
from os.path import join as pjoin
import matplotlib.pyplot as plt
from spks.event_aligned import population_peth
from spks.utils import alpha_function

pd.set_option("display.max_columns", 100)

plt.rcParams["text.usetex"] = False
plt.rcParams["svg.fonttype"] = "none"
plt.rcParams["font.sans-serif"] = "Arial"
plt.rcParams["font.size"] = 12
plt.rcParams["figure.dpi"] = 100

%config InlineBackend.figure_format = 'svg'
%matplotlib widget
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
# %% Load data
animal = "GRB006"  # example animal
session = "20240723_142451"  # example session

data_dir = "/Users/gabriel/data"
trial_ts = pd.read_pickle(
    pjoin(data_dir, animal, session, "pre_processed", "trial_ts.pkl")
)
spike_times_per_unit = np.load(
    pjoin(data_dir, animal, session, "pre_processed", "spike_times_per_unit.npy"),
    allow_pickle=True,
)

trial_ts = trial_ts[
    trial_ts["stationary_stims"].apply(lambda x: len(x) > 0)
    & trial_ts["movement_stims"].apply(lambda x: len(x) > 0)
    & trial_ts["center_port_entries"].apply(lambda x: len(x) > 0)
].copy()

In [45]:
first_stim_ts = trial_ts["first_stim_ts"].to_numpy(dtype=float).copy()

binwidth_ms = 10
t_decay = 0.025
t_rise = 0.001
decay = t_decay / (binwidth_ms / 1000)
kern = alpha_function(
    int(decay * 15), t_rise=t_rise, t_decay=decay, srate=1.0 / (binwidth_ms / 1000)
)

first_stim_peth, bin_edges, event_index = population_peth(
    all_spike_times=spike_times_per_unit,
    alignment_times=first_stim_ts,
    pre_seconds=0.1,
    post_seconds=0.15,
    binwidth_ms=binwidth_ms,
    kernel=kern,
)

In [46]:
first_stim_peth.shape

(137, 381, 23)